<a href="https://colab.research.google.com/github/WaterPurify/Alpaco_Bootcamp/blob/main/2_Natural%20Language%20%26%20Speech%20Recognition/Team_project/Code/fasterWhisper_w_PEFT_finetune_version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA (Low Rank Adaptation) 와 PEFT (Paramater Efficient Fine Tuning)를 적용한 Larger Whisper fine-tuning ⚡️

* 소비자 GPU의 VRAM이 8GB 미만인 환경에서도 full-finetuning과 유사한 성능을 제공함
*  🤗 Transformers and PEFT 모델과 Common Voice 13.0 dataset를 사용하여 Whisper fine-tune하는 과정 설명함
* PEFT와 bitsandbytes를 활용하여 무료 T4 GPU(16GB VRAM)를 사용하여 whisper-large-v2 체크포인트를 원활하게 학습

## 왜 Parameter Efficient Fine Tuning [PEFT](https://github.com/huggingface/peft)를 사용해야 되는가?


* 모델 사이즈 증가로 fine tuning하는 것이 계산 복잡성 증가와 메모리 사용량 증가
    * 예를 들어, Whisper-large-v2 모델을 완전한 미세 조정을 위해 약 24GB의 GPU VRAM이 필요하며, 각 미세 조정된 모델은 약 7GB의 저장 공간을 필요함

    * 제한적인 환경에서 bottleneck 발생하고 원하는 결과를 얻기 힘듦
* PEFT
    * 효과적으로 parameter를 줄여서 fine tuning 속도 개선
    * 목적: 병목 현상을 해결
    * 접근법(예: 저수준 적응): 사전 훈련된 모델의 대부분의 매개변수를 동결시키면서 추가 모델 매개변수의 일부만 미세 조정하여 계산 및 저장 비용 크게 줄임
        * 대규모 모델의 전체 미세 조정 중 관찰되는 catastrophic forgetting 문제를 극복할 수 있음

### LoRA가 무엇인가?

* PEFT에서 여러 매개변수 효율적인 기술을 기본으로 제공
    * 그 중 하나인 Low Rank Adaptation (LoRA)
        * 사전 훈련된 모델 가중치를 동결하고 Transformer 아키텍처의 각 레이어에 훈련 가능한 랭크 분해 행렬을 삽입 (High Rank 즉 많은 연결이 되어 있는 것들보다 연결이 적은 Low Rank로 만들어서 계산량을 줄임)
            * Downstream 작업에 대한 훈련 가능한 매개변수 수가 크게 감소

### 통계로 보는 PEFT 효과

* Full fine-tuning of Whisper-large-v2 checkpoint Vs. PEFT 적용 모델

    1. GPU VRAM이 8GB 미만인 환경에서 16억 개의 매개변수를 가진 모델을 미세 조정 🤯
    2. 훨씬 적은 수의 훈련 가능한 매개변수를 사용하여 거의 5배 더 큰 배치 크기를 사용 가능 📈
    3. 생성된 체크포인트는 원본 모델의 크기의 1%인 약 60MB 🚀
* 기존 🤗 transformers Whisper에서 변형이 많이 되지 않았음

    





## 환경설정


* Python package->Whisper 모델 fine tuning하기 위해 사용
  * `datasets`:학습 데이터를 다운로드하고 준비
  * `transformers`: Whisper 모델을 로드하고 훈련
  * `librosa`: 오디오 파일을 전처리
  * `evaluate` &  `jiwer`:모델의 성능을 평가
  * `PEFT`, `bitsandbytes`, `accelerate`: LoRA로 모델과 fine-tuning

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main


* GPU 확보
    * Google Colab Pro: V100 또는 P100 GPU가 할당
* GPU 확보 방법
    * 런타임 -> 런타임 유형 변경
    * None -> GPU 변경
* GPU 할당 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 26 09:50:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0             33W /  135W |       0MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Colab에서 제공하는 GPU사용

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!


* 학습 중 모델 체크포인트를 직접 Hugging Face Hub에 업로드하는 것을 강력히 권장
 * Hub를 사용하면 다음과 같은 기능을 제공:

 1. 통합된 버전 관리: 훈련 중에 어떠한 모델 체크포인트도 손실되지 않음
 2. Tensorboard 로그: 훈련 과정에서 중요한 지표를 추적
 3. 모델 카드: 모델이 무엇을 하고 의도된 사용 사례를 문서화
 4. 커뮤니티: 커뮤니티와 쉽게 공유하고 협업

* Hub에 연결
 * 프롬프트에서 Hub 인증 토큰을 입력:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Whisper 모델 checkpoint와 task 설정

In [ ]:
model_name_or_path = "openai/whisper-large-v2"
task = "transcribe"



데이터셋 디테일을 설정 (언어)

In [ ]:
dataset_name = "mozilla-foundation/common_voice_13_0"
language = "Korean"
language_abbr = "ko" # Short hand code for the language we want to fine-tune

# 데이터셋 올리기

* Huggingface Dataset 사용
 * 적은 코드로 Common Voice의 데이터를 다운로드하고 준비

* 확인 절차
 1. Hugging Face Hub의 이용 약관을 수락했는지 확인:[mozilla-foundation/common_voice_13_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0)
 2. 데이터셋에 액세스하고 로컬로 데이터를 다운로드

* 학습+검증 데이터셋/테스트 데이터셋 분리

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation", token=True)
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test", token=True)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 297
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 131
    })
})


* 일반적인 ASR(음성 인식) 데이터셋
    * 입력 오디오 샘플(오디오)과 해당되는 텍스트(문장)만 제공
* Common Voice
    * ASR에는 필요하지 않은 악센트와 로케일과 같은 추가 메타데이터 정보가 포함
    * 일반적인 용도로 사용하고 미세 조정을 고려하기 위해 메타데이터 정보 무시

In [ ]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 297
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 131
    })
})


## 특성 추출기(Feature Extractor), 토크나이저(Tokenizer), 그리고 데이터준비

ASR 파이프라인 세 단계로 분해:

1. Raw 오디오 입력을 전처리하는 특정 추출기
2. 시퀀스 간 매핑을 수행하는 모델
3. 모델 출력을 텍스트 형식으로 후처리하는 tokenizer


* Whisper
    * [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)와 [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)로 구성



In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


* WhisperProcessor 클라스
    * 특성 추출기와 토크나이저를 사용하기 위해 두 가지를 모두 합칩
    * 필요에 따라 오디오 입력 및 모델 예측에 사용 가능

* 학습 중에 두 개의 객체만 추적 필요: 프로세서와 모델

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 데이터 준비

Common Voice 데이터셋의 첫 번째 예제를 출력하여 데이터의 형식을 살펴봄

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': 'C:\\Users\\User\\.cache\\huggingface\\datasets\\downloads\\extracted\\f4cce98295bb5b9d65d17d7d495bef59577e50b412400821477bb56f2c098300\\ko_train_0/common_voice_ko_35845802.mp3', 'array': array([-4.54747351e-13,  4.77484718e-12,  6.59383659e-12, ...,
        2.07564299e-05,  1.92765128e-06, -1.09442644e-05]), 'sampling_rate': 48000}, 'sentence': '어느덧 그 더운 팔월도 하루를 남기고 다 지나 버렸다.'}


* Whisper 모델 샘플링
    * 입력 오디오는 48 kHz 새플링
    * Whisper feature extractor에 전달하기 위해서 16 kHz로 다운샘플 진행
* 샘플링 속도 설정
    * Dataset의 [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column) 방법 사용: 오디오 입력을 올바른 샘플링 속도로 설정
    * 오디오를 변경하는 것이 아니라 오디오 샘플을 실시간으로 받을 수 있도록 함


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


Common Voice 데이터셋에서 첫 번째 오디오 샘플을 다시로드하면 원하는 샘플링 속도로 다시 샘플링

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': 'C:\\Users\\User\\.cache\\huggingface\\datasets\\downloads\\extracted\\f4cce98295bb5b9d65d17d7d495bef59577e50b412400821477bb56f2c098300\\ko_train_0/common_voice_ko_35845802.mp3', 'array': array([ 1.45519152e-11,  0.00000000e+00,  2.18278728e-11, ...,
        4.60762531e-05,  3.03988345e-05, -1.03190541e-06]), 'sampling_rate': 16000}, 'sentence': '어느덧 그 더운 팔월도 하루를 남기고 다 지나 버렸다.'}



* 모델에 맞게 데이터를 준비하는 함수:

1. batch["audio"]를 호출하여 오디오 데이터를 로드하고 다시 샘플링. 🤗 Datasets는 필요한 모든 재샘플링 작업을 실시간으로 수행
3. Feature extractor를 사용하여 1차원 오디오 배열에서 로그 멜 스펙트로그램 입력 특성을 계산
3. Tokenizer를 사용하여 transcripts를 레이블 ids로 인코딩


In [ ]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]

#     # compute log-Mel input features from input audio array
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

#     # encode target text to label ids
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch

def prepare_dataset(batch, feature_extractor, tokenizer):
    # Load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # Compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # Encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


* 데이터 준비 함수
    * 데이터셋의 .map 메서드를 사용하여 모든 학습 예제에 적용 가능
    * num_proc: 몇 개의 CPU 코어를 사용할 지를 지정,num_proc를 1보다 크게 설정하면 다중 처리가 활성화 (다중 처리로 .map 메서드가 중단되는 경우 num_proc=1로 설정하고 데이터셋을 순차적으로 처리)

* Dataset의 사이즈에 따라서 20~30 정도 걸림


In [ ]:
# common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

common_voice = common_voice.map(
    prepare_dataset,
    remove_columns=common_voice.column_names["train"],
    num_proc=2,
    fn_kwargs={"feature_extractor": feature_extractor, "tokenizer": tokenizer}
)


Map (num_proc=2):   0%|          | 0/297 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/131 [00:00<?, ? examples/s]

In [ ]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 297
})

## 학습 및 검증


* 훈련 파이프라인
* [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)가 대부분의 작업을 처리:


1. 데이터 collator 정의: 데이터 콜레이터는 우리가 전처리한 데이터를 가져와 모델에 사용할 수 있는 PyTorch 텐서로 준비
2. 평가 지표: 평가 중에는 모델을 글자 오류율  [word error rate (CER)](https://huggingface.co/metrics/cer)지표를 사용하여 평가
3. 사전 훈련된 체크포인트 load: 사전 훈련된 체크포인트를 로드하고 훈련을 위해 올바르게 구성
4. 훈련 구성 정의: 🤗 Trainer가 훈련 스케줄을 정의에 사용

* 미세 조정한 후에는 테스트 데이터에서 모델을 평가하여 한국어 음성을 올바르게 transcribe


### Data Collator 정의

* 시퀀스-투-시퀀스 음성 모델의 데이터 콜레이터
    * Input_features와 labels를 독립적으로 처리 차별화
    
    - Input_features: feature extractor에 의해 처리
    - labels: tokenizer에 의해 처리

* Input_features는 이미 30초로 패딩되어 있고 특성 추출기에 의해 고정된 차원의 로그 멜 스펙트로그램으로 변환. 따라서 우리가 해야 할 일은 input_features를 배치 처리된 PyTorch 텐서로 변환

* labels는 패딩되지 않음 먼저 배치 내에서 최대 길이에 맞게 시퀀스를 패딩하고, tokenizer의 .pad 메서드를 사용하여 시퀀스를 패딩 패딩 토큰은 손실을 계산할 때 고려되지 않도록 -100으로 대체. 그런 다음 레이블 시퀀스의 시작에서 BOS 토큰을 잘라서 훈련 중에 나중에 이를 추가

* 이전에 정의한 WhisperProcessor를 활용하여 특성 추출기 및 토크나이저 작업을 모두 수행 가능

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Data collator 초기화 진행

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### 평가 지표


* ASR 시스템을 평가하기 위한 '사실상의' 지표인 한 단어 오류율(CER) 메트릭을 사용
* 더 많은 정보는 [문서](https://huggingface.co/metrics/cer)를 참조. 우리는 🤗 Evaluate에서 CER 메트릭을 로드

In [ ]:
import evaluate

metric = evaluate.load("cer")

### Pre-trained 모델 로드

* 사전 훈련된 Whisper 체크포인트를 로드
    * 이 작업은 🤗 Transformers를 사용하여 매우 간단



* 모델의 메모리 사용량을 줄이기 위해 모델을 8비트로         
    * 모델을 1/4 정밀도(32비트와 비교했을 때)로 양자화하여 성능 손실을 최소화 [here](https://huggingface.co/blog/hf-bitsandbytes-integration)

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map={"":0})

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


### 모델의 후처리

1. 훈련을 가능하게 하기 위해 8비트 모델에 몇 가지 후처리 단계를 적용
2. 모델 레이어를 동결, 훈련과 모델의 안정성을 위해 레이어 정규화와 출력 레이어를 float32로 캐스팅

(모델 안정성과 layer normalization 분석, float32로 캐스팅 하는 이유)

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)


* Whisper 모델은 인코더에 컨볼루션 레이어를 사용하기 때문에 체크포인팅은 grad 연산을 비활성. 이를 피하기 위해 입력을 특별히 trainable하게 만들어야 합니다.


In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

### Low-rank adapters (LoRA)를 모델에 적용

* peft에서 get_peft_model 유틸리티 함수를 사용하여 PeftModel을 로드하고 저희가 저차원 어댑터(LoRA)를 사용할 것임을 지정


In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 15,728,640 || all params: 1,559,033,600 || trainable%: 1.0089


**1%**의 학습 parameter를 사용하였고 **Parameter-Efficient Fine-Tuning**를 적용


### 훈련 구성 정의


마지막 단계에서는 훈련과 관련된 모든 매개변수를 정의 훈련 인자에 대한 자세한 내용은 해당 문서를 참조 Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="reach-vb/test",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=1,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=100,
    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)


* PEFT를 사용하여 모델을 미세 조정하는 것에는 몇 가지 주의가 필요

1. PeftModel의 forward가 기본 모델의 forward의 시그니처를 상속하지 않기 때문에 remove_unused_columns=False 및 label_names=["labels"]를 명시적으로 설정
2.INT8 훈련에는 자동 캐스팅이 필요하기 때문에 Trainer에서 기본적으로 제공되는 predict_with_generate 호출을 사용할 수 없습니다. 자동 캐스팅이 자동으로 적용되지 않음
3. 자동 캐스팅을 사용할 수 없으므로 Seq2SeqTrainer에 compute_metrics를 전달할 수 없음. 따라서 Trainer를 인스턴스화하는 동안 compute_metrics를 주석 처리 필요



In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

C:\AEO\anaconda3\envs\gpuconda\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\AEO\anaconda3\envs\gpuconda\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
C:\AEO\anaconda3\envs\gpuconda\lib\site-packages\transformers\models\whisper\modeling_whisper.py:694: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transform

Step,Training Loss,Validation Loss
100,0.541300,0.231048


TrainOutput(global_step=100, training_loss=0.5412639236450195, metrics={'train_runtime': 1017.2357, 'train_samples_per_second': 0.786, 'train_steps_per_second': 0.098, 'total_flos': 1.6866147262464e+18, 'train_loss': 0.5412639236450195, 'epoch': 2.6315789473684212})

Fine-tuning한 모델을 Hugging Face Hub에 저장합니다. 나중에 모델을 불러올 때 편합니다.

In [ ]:
peft_model_id = "ZeroWater93/whisper-large-v2-korea-common_13"
model.push_to_hub(peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ZeroWater93/whisper-large-v2-korea-common_13/commit/71992e7d98bf7aaa1135f11a84742d63d4b80df3', commit_message='Upload model', commit_description='', oid='71992e7d98bf7aaa1135f11a84742d63d4b80df3', pr_url=None, pr_revision=None, pr_num=None)

# 평가 및 검증

Finetuning을 성공적으로 했으면 이제 테스트 데이터셋에서 저희 모델을 테스트하고 CER(Character Error Rate) 계산

테스트 유의할 점들:

1. predict_with_generate 함수를 사용할 수 없으므로 자체적으로 torch.cuda.amp.autocast()를 사용하여 eval 루프를 구현

2. 기본 모델이 동결되어 있기 때문에 PEFT 모델은 때로 디코딩 중에 언어를 인식하지 못할 수 있음. 이를 해결하기 위해 디코딩 시작 토큰에 번역 중인 언어를 명시적으로 지정. 이 작업은 forced_decoder_ids = processor.get_decoder_prompt_ids(language="Marathi", task="transcribe")를 사용하여 수행하고 model.generate 호출에 이를 전달

Transcribe를 진행 🔥


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "ZeroWater93/whisper-large-v2-korea-common_13" # Use the same model ID as before.
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

adapter_config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

 35%|█████████████████████████████▎                                                     | 6/17 [01:37<02:59, 16.35s/it]

## 마무리!

Whisper 체크포인트를 더 빠르고 저렴하게 훈련하고 CER에서 거의 손실이 없도록 학습하는 방법을 배움

PEFT (Pretraining Efficiently with Fine-Tuning)를 사용하면 음성 인식 이외에도 다른 사전 훈련된 모델에 동일한 기술 세트를 적용 가능. 아래 링크에서 자세히 설명: https://github.com/huggingface/peft 🤗